# PKE - Wymiana klucza Diffiego-Hellmana

## Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę* $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{k}&\text{ dla }k<0
\end{array}\right.$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$, *logarytmem dyskretnym* $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

## Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\mod n$, o ile w ogóle istnieje.

Specyficzną sytuacją w teorii liczb jest gdy $n=p$ jest liczbą pierwszą a $q$ jest pierwiastkiem pierwotnym $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest generatorem grupy multiplikatywneh rzędu $p-1$
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$

## Algorytm baby-step giant-step

Jeden z najprostszych (poza metodą naiwną) algorytmów poszukiwania logarytmu dyskretnego w grupach cyklicznych.

Niech $p$ będzie liczbą pierwszą oraz niech $q$ będzie pierwiastkiem pierwotnym modulo $p$. Dla niezerowego $a\in\mathbb{Z}_p$ szukamy liczby $k\in\mathbb{Z}$ takiej, że $q^k=a\mod p$

### Krok 1.
- $m=\lceil\sqrt{p-1}\rceil$
- tworzymy pomocniczą tablicę potęg: dla wszystkich $i\in [0,m)$ obliczamy parę $(i,q^i)$
- obliczamy $r=(q^{-1})^m$
### Krok 2.
- $b=a$
- dla wszystkich $j\in [0,m)$:
    - sprawdzamy, czy para $(i,b)$ jest elementem tablicy potęg dla pewnego $i$
    - jeżeli tak, to $k=jm+i$ i kończymy algorytm
    - jeżeli nie, to $b=br$ i kontynuujemy pętlę


In [1]:
import math
import random
import time

## Zadanie 1.

Zaimplementować algorytm baby-step giant-step. Przetestować dla podanych danych testujących.

```Dane testujące:
p = 7
q = 3
a = 4

m = 3
tablica_testowa = [1,3,2]
r = 6
k = 4 (j = 1, i = 1)
```

```
p = 29
q = 8
a = 10

m = 6
tablica_testowa = [1,8,6,19,7,27]
r = 9
k = 17 (j = 2, i = 5)
```

```
p = 113
q = 76
a = 84

m = 11
tablica_testowa = [1,76,13,84,56,75,50,71,85,19,88]
r = 70
k = 3 (j = 0, i = 3)
```

In [2]:
def baby_step_giant_step(p, q, a):
    m = math.ceil(math.sqrt(p - 1))
    table = {pow(q, i, p): i for i in range(m)}  # Baby steps

    q_inv_m = pow(q, -m, p)
    b = a

    for j in range(m):
        if b in table:
            return j * m + table[b]
        b = (b * q_inv_m) % p

    return None  # No solution found

# Example tests
print(baby_step_giant_step(7, 3, 4))   # Expected: 4
print(baby_step_giant_step(29, 8, 10)) # Expected: 17
print(baby_step_giant_step(113, 76, 84)) # Expected: 3

4
17
3


## Algorytm wymiany klucza Diffiego-Hellmana

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$
- Alice generuje $x=q^n\mod p$ i wysyła do Boba
- Bob generuje $y=q^m\mod p$ i wysyła Alice
- Alice oblicza klucz $k=y^n\mod p$
- Bob oblicza klucz $k=x^m\mod p$


## Zadanie 2.

Zaimplementuj powyższy algorytm wymiany klucza.

In [3]:
def diffie_hellman(p, q):
    # Alice and Bob select private secrets
    alice_secret = random.randint(1, p-1)
    bob_secret = random.randint(1, p-1)

    # Alice computes x and sends to Bob
    x = pow(q, alice_secret, p)

    # Bob computes y and sends to Alice
    y = pow(q, bob_secret, p)

    # Both compute the shared secret
    alice_key = pow(y, alice_secret, p)
    bob_key = pow(x, bob_secret, p)

    assert alice_key == bob_key  # The shared secret must match
    return alice_key, alice_secret, bob_secret, x, y

# Test example
p, q = 23, 5
shared_key, alice_secret, bob_secret, x, y = diffie_hellman(p, q)

print(f"Shared Key: {shared_key}")
print(f"Alice Secret: {alice_secret}, Alice sends: {x}")
print(f"Bob Secret: {bob_secret}, Bob sends: {y}")

Shared Key: 1
Alice Secret: 4, Alice sends: 4
Bob Secret: 22, Bob sends: 1


## Zadanie 3.
Na swoją własną implementację przeprowadź atak algorytmem baby-step, giant-step. Jeżeli atak zakónczył się sukcesem, zmodyfikuj parametry $p$ i $q$ tak, żeby atak algorytmem z zadania 1 nie zakończył się sukcesem w rozsądnym czasie.

In [6]:
def baby_step_giant_step_with_timeout(p, q, a, timeout_seconds=60):
    m = math.isqrt(p) + 1
    baby_steps = {}

    start_time = time.time()

    # Building baby_steps dictionary with a periodic timeout check
    for i in range(m):
        if time.time() - start_time > timeout_seconds:
            print("Timeout exceeded during setup (baby steps).")
            return None
        baby_steps[pow(q, i, p)] = i

    q_inv_m = pow(q, -m, p)
    gamma = a

    for j in range(m):
        if time.time() - start_time > timeout_seconds:
            print("Timeout exceeded during search (giant steps).")
            return None
        if gamma in baby_steps:
            return j * m + baby_steps[gamma]
        gamma = (gamma * q_inv_m) % p

    return None

# Recommended practical Diffie-Hellman parameters (secure but manageable)
p = 1000000000000019   # prime ~10^15, securely large yet practical for testing
q = 2                  # primitive root mod p

def secure_diffie_hellman_final(p, q):
    alice_secret = random.randint(1, p - 1)
    bob_secret = random.randint(1, p - 1)

    x = pow(q, alice_secret, p)
    y = pow(q, bob_secret, p)

    alice_key = pow(y, alice_secret, p)
    bob_key = pow(x, bob_secret, p)

    assert alice_key == bob_key

    return alice_secret, bob_secret, x, y, alice_key

alice_secret, bob_secret, x, y, alice_key = secure_diffie_hellman_final(p, q)

print(f"Diffie-Hellman parameters:\np = {p}\nq = {q}\nx = {x}")

# Attempt the attack with timeout (set to 60 seconds)
start_time = time.time()
recovered_secret = baby_step_giant_step_with_timeout(p, q, x, timeout_seconds=60)
end_time = time.time()

if recovered_secret is None:
    print("Attack failed or timeout reached: complexity too high.")
else:
    print(f"Attack succeeded unexpectedly: {recovered_secret}")

print(f"Attack attempt took {end_time - start_time:.2f} seconds.")

Diffie-Hellman parameters:
p = 1000000000000019
q = 2
x = 668454762283538
Timeout exceeded during setup (baby steps).
Attack failed or timeout reached: complexity too high.
Attack attempt took 60.51 seconds.
